# University Course Timetabling with Simulated Annealing

This notebook demonstrates the use of **Simulated Annealing (SA)** for solving the university course timetabling problem - a complex constraint satisfaction problem (CSP).

## What is Simulated Annealing?

Simulated Annealing is a probabilistic optimization technique inspired by the annealing process in metallurgy. It's particularly effective for:
- Large combinatorial optimization problems
- Problems with many local optima
- Constraint satisfaction problems

## Key Features of this Implementation

- **Multi-phase optimization**: First eliminate hard constraint violations, then optimize soft constraints
- **Tabu Search**: Prevents cycling back to recently visited solutions
- **Intensification phase**: Targeted search for remaining hard violations
- **Reheating**: Periodically increases temperature to escape local optima
- **Multiple move operators**: Diversified search strategies

## Problem Constraints

### Hard Constraints (must be satisfied)
- No room conflicts (one class per room at a time)
- No lecturer conflicts (one lecturer teaching at a time)
- No class conflicts (no overlapping classes for same program)
- Room capacity must accommodate class size
- Friday prayer time restrictions
- Class type time matching (morning classes in morning slots)

### Soft Constraints (optimization goals)
- Schedule compactness (minimize gaps)
- Preferred room/time assignments
- Research day preferences
- Minimize prayer time overlaps
- Proper room type usage

## 1. Setup and Imports

First, we install the package in development mode and import all necessary modules.

In [29]:
# Install package in development mode
%pip install pip install git+https://github.com/albertabayor/timetable-sa.git@feature/python-port#egg=timetable-sa&subdirectory=python


# Import standard libraries
import sys
import copy
import json
import random
import time as time_python

# Import pandas for data display
import pandas as pd

# Core framework
from timetable_sa import SimulatedAnnealing, SAConfig
from timetable_sa.core.interfaces.config import LoggingConfig

# Domain types
from timetable_sa.examples.timetabling.domain_types.domain import (
    Room, Lecturer, TimeSlot, ClassRequirement
)
from timetable_sa.examples.timetabling.domain_types.state import (
    ScheduleEntry, TimetableState
)

# All constraints and moves
from timetable_sa.examples.timetabling import (
    # Hard constraints
    NoRoomConflict, NoLecturerConflict, NoProdiConflict, RoomCapacity,
    MaxDailyPeriods, FridayTimeRestriction, NoFridayPrayConflict,
    PrayerTimeStart, ClassTypeTime, SaturdayRestriction,
    # Soft constraints
    Compactness, OverflowPenalty, PreferredRoom, PreferredTime,
    TransitTime, ResearchDay, PrayerTimeOverlap, EveningClassPriority,
    # Move generators
    ChangeTimeSlot, ChangeRoom, SwapClasses, ChangeTimeSlotAndRoom,
    FixRoomConflict, FixLecturerConflict, FixRoomCapacity,
    FixMaxDailyPeriods, FixFridayPrayerConflict, FixClassTypeTime,
    SwapFridayWithNonFriday,
)

# Utilities
from timetable_sa.examples.timetabling.utils.time import (
    time_to_minutes, calculate_end_time,
)
from timetable_sa.examples.timetabling.data.excel_loader import load_uisi_data

print("✅ All imports successful!")

Note: you may need to restart the kernel to use updated packages.
✅ All imports successful!


## 2. Configuration Parameters

Adjust these parameters to customize the optimization behavior.

In [25]:
# Data file path
EXCEL_DATA_PATH = "/home/emmanuelabayor/projects/timetable-sa/data_uisi.xlsx"

# SA Parameters
INITIAL_TEMPERATURE = 100000.0
MIN_TEMPERATURE = 0.0000001
COOLING_RATE = 0.9995
MAX_ITERATIONS = 100000

# Tabu Search (prevents cycling)
TABU_SEARCH_ENABLED = True
TABU_TENURE = 500  # Number of iterations to remember a state

# Reheating (helps escape local optima)
REHEATING_THRESHOLD = 2000  # Iterations without improvement before reheating
REHEATING_FACTOR = 2.0  # Temperature multiplier when reheating
MAX_REHEATS = 3  # Maximum number of reheating events

# Intensification (targeted search for hard violations)
ENABLE_INTENSIFICATION = True
INTENSIFICATION_ITERATIONS = 2000
MAX_INTENSIFICATION_ATTEMPTS = 3

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

print("⚙️  Configuration loaded!")
print(f"   Temperature: {INITIAL_TEMPERATURE} → {MIN_TEMPERATURE}")
print(f"   Cooling rate: {COOLING_RATE}")
print(f"   Max iterations: {MAX_ITERATIONS}")
print(f"   Tabu search: {TABU_SEARCH_ENABLED} (tenure={TABU_TENURE})")

⚙️  Configuration loaded!
   Temperature: 100000.0 → 1e-07
   Cooling rate: 0.9995
   Max iterations: 100000
   Tabu search: True (tenure=500)


## 3. Load Data from Excel

Load the university data including rooms, lecturers, and class requirements.

In [36]:
print("=" * 60)
print("University Course Timetabling - Jupyter Notebook")
print("=" * 60)

# Load data
rooms, lecturers, classes = load_uisi_data(EXCEL_DATA_PATH)

# Display summary
print(f"\n📊 Data Summary:")
print(f"   Rooms: {len(rooms)}")
print(f"   Lecturers: {len(lecturers)}")
print(f"   Classes: {len(classes)}")

# Display sample data
print(f"\n🏠 Sample Rooms (first 5):")
df_rooms = pd.DataFrame([{
    'Code': r.code, 'Name': r.name, 'Type': r.type, 'Capacity': r.capacity
} for r in rooms[:5]])
display(df_rooms.style.hide(axis='index'))

print(f"\n👨‍🏫 Sample Lecturers (first 5):")
df_lecturers = pd.DataFrame([{
    'Code': l.code, 'Name': l.name, 'Prodi': l.prodi
} for l in lecturers[:5]])
display(df_lecturers.style.hide(axis='index'))

print(f"\n📚 Sample Classes (first 5):")
df_classes = pd.DataFrame([{
    'Code': c.kode_matakuliah, 'Name': c.mata_kuliah, 'Class': c.kelas,
    'SKS': c.sks, 'Participants': c.peserta
} for c in classes[:5]])
display(df_classes.style.hide(axis='index'))

University Course Timetabling - Jupyter Notebook

📊 Data Summary:
   Rooms: 33
   Lecturers: 99
   Classes: 373

🏠 Sample Rooms (first 5):


Code,Name,Type,Capacity
B2-R1,Kampus B B2 Ruang 1,theory,30
B3-R1,Kampus B B3 Ruang 1,theory,30
B3-R2,Kampus B B3 Ruang 2,theory,30
B3-R3,Kampus B B3 Ruang 3,theory,50
CM-101,Kampus B Gedung 1 Lantai 1 Ruang 1,theory,45



👨‍🏫 Sample Lecturers (first 5):


Code,Name,Prodi
RPA,"Dr. Rr. Rooswanti Putri Adi Agustini, S.Kom., M.M",Magister Management
GTK,"Dr. Ir. Gatot Kustyadji, S.E., M.Si., IPU., ASEAN Eng., APEC Eng.",Magister Management
ALF,"Dr. Alfina, S.M., M.M.",Magister Management
BAT,"Dr. Bambang Tutuko, S.E., M.M., CFP®",Magister Management
ANW,"Aditya Narendra Wardhana, S.T., M.SM.",Management



📚 Sample Classes (first 5):


Code,Name,Class,SKS,Participants
MM23EB03,Economic for Business,MM-1A,3,15
MM23SM03,Strategic Marketing Management,MM-1A,3,15
MM23HC03,Strategic Human Capital and Change Management,MM-1A,3,15
MM23CF03,Corporate Finance,MM-1A,3,15
MM23OS03,Operation and Supply Chain Management,MM-1A,3,15


## 4. Generate Time Slots

Create time slots for morning (PAGI) and evening sessions. The university has specific time periods:
- **Morning (PAGI)**: 07:30 - 15:30
- **Evening**: 18:00 - 21:00 (only evening slots from SORE period)

In [41]:
# Import from library
from timetable_sa.examples.timetabling.utils.timeslot_generator import generate_ts_slots
from timetable_sa.examples.timetabling.utils.time import time_to_minutes

# Generate slots using library function
pagi_slots = generate_ts_slots("07:30", "15:30", 50)
sore_slots = generate_ts_slots("15:30", "21:00", 50)
evening_slots = [s for s in sore_slots if time_to_minutes(s.start_time) >= time_to_minutes("18:00")]
time_slots = pagi_slots + evening_slots

print(f"\n⏰ Time Slots Generated:")
print(f"   Morning (PAGI): {len(pagi_slots)} slots")
print(f"   Evening: {len(evening_slots)} slots")
print(f"   Total: {len(time_slots)} slots")

# Show sample time slots
print(f"\n📅 Sample Monday Time Slots:")
monday_slots = [s for s in time_slots if s.day == 'Monday'][:10]
for slot in monday_slots:
    print(f"   Period {slot.period}: {slot.start_time} - {slot.end_time}")


⏰ Time Slots Generated:
   Morning (PAGI): 54 slots
   Evening: 18 slots
   Total: 72 slots

📅 Sample Monday Time Slots:
   Period 1: 07:30 - 08:20
   Period 2: 08:20 - 09:10
   Period 3: 09:10 - 10:00
   Period 4: 10:00 - 10:50
   Period 5: 10:50 - 11:40
   Period 6: 11:40 - 12:30
   Period 7: 12:30 - 13:20
   Period 8: 13:20 - 14:10
   Period 9: 14:10 - 15:00
   Period 4: 18:00 - 18:50


## 5. Create Initial State

Use a greedy algorithm to create the initial timetable. This algorithm:
1. Shuffles classes for random placement order
2. For each class, finds suitable rooms based on capacity and lab requirements
3. Checks for conflicts (room, lecturer, prodi/class)
4. Places classes without conflicts

In [42]:
# Import from library
from timetable_sa.examples.timetabling.utils.initial_solution import create_greedy_initial_state_v2

print("\n🔨 Creating Initial State (Greedy Algorithm)...")
state = create_greedy_initial_state_v2(classes, rooms, lecturers, time_slots, RANDOM_SEED)

print(f"   Schedule entries: {len(state.schedule)}")
print(f"   Available rooms: {len(state.rooms)}")
print(f"   Available lecturers: {len(state.lecturers)}")

# Show sample schedule entries
print(f"\n📋 Sample Schedule Entries (first 5):")
for entry in state.schedule[:5]:
    print(f"   {entry.class_id}: {entry.class_name}")
    print(f"      {entry.time_slot.day} {entry.time_slot.start_time}-{entry.time_slot.end_time}")
    print(f"      Room: {entry.room}, Lecturers: {entry.lecturers}")


🔨 Creating Initial State (Greedy Algorithm)...
   Placed: 356/373
   Skipped: GS13TH46: No lecturers
   Skipped: GS13PW02: No lecturers
   Skipped: GS13IP12: No lecturers
   Skipped: CE11UT46: No lecturers
   Skipped: GS13PW02: No lecturers
   Skipped: AC135343: No lecturers
   Skipped: GS13CZ02: No lecturers
   Skipped: GS13CZ02: No lecturers
   Skipped: GS13IP12: No lecturers
   Skipped: GS13CZ02: No lecturers
   ... and 7 more
   No conflicts in initial state
   Schedule entries: 356
   Available rooms: 33
   Available lecturers: 99

📋 Sample Schedule Entries (first 5):
   VD13GB03: Gambar Bentuk
      Monday 07:30-10:00
      Room: G4-R2, Lecturers: ['MNR']
   IS13ST03: Pengantar Sistem & Teknologi Informasi
      Monday 07:30-10:00
      Room: CM-203, Lecturers: ['TKN']
   ET13EE13: EKONOMI TEKNIK
      Monday 07:30-10:00
      Room: CM-201, Lecturers: ['LKT']
   CE11TH23: Termodinamika Teknik Kimia 1
      Monday 07:30-10:00
      Room: CM-207, Lecturers: ['YNK']
   LE12AJ53: Ma

## 6. Define Constraints

Constraints define the rules that a valid timetable must follow.

In [14]:
# Hard constraints (MUST be satisfied)
hard_constraints = [
    NoRoomConflict(),           # No two classes in same room at same time
    NoLecturerConflict(),       # No lecturer teaching two classes simultaneously
    NoProdiConflict(),          # No overlapping classes for same program
    RoomCapacity(),             # Room capacity must accommodate class size
    MaxDailyPeriods(),          # Limit classes per day
    FridayTimeRestriction(),    # Specific time restrictions for Friday
    NoFridayPrayConflict(),     # Avoid prayer times on Friday
    PrayerTimeStart(),          # Start times must avoid prayer periods
    ClassTypeTime(),            # Morning classes in pagi slots, evening in sore
    SaturdayRestriction(),      # Non-MM programs can't use Saturday
]

# Soft constraints (optimization goals)
soft_constraints = [
    Compactness(),              # Minimize gaps in schedule
    OverflowPenalty(),          # Penalty for using lab rooms for non-lab classes
    PreferredRoom(),            # Preference for specific rooms
    PreferredTime(),            # Preference for specific times
    TransitTime(),              # Minimize time between classes
    ResearchDay(),              # Preferred research days for lecturers
    PrayerTimeOverlap(),        # Avoid prayer time overlaps
    EveningClassPriority(),     # Prioritize evening classes in sore slots
]

all_constraints = hard_constraints + soft_constraints

print(f"\n📋 Constraints:")
print(f"   Hard: {len(hard_constraints)}")
for c in hard_constraints:
    print(f"      - {c.name}")
print(f"   Soft: {len(soft_constraints)}")
for c in soft_constraints:
    print(f"      - {c.name}")


📋 Constraints:
   Hard: 10
      - No Room Conflict
      - No Lecturer Conflict
      - No Prodi Conflict
      - Room Capacity
      - Max Daily Periods
      - Friday Time Restriction
      - No Friday Prayer Conflict
      - Prayer Time Start
      - Class Type Time
      - Saturday Restriction
   Soft: 8
      - Compactness
      - Overflow Penalty
      - Preferred Room
      - Preferred Time
      - Transit Time
      - Research Day
      - Prayer Time Overlap
      - Evening Class Priority


## 7. Define Move Generators

Move generators are operators that modify the current solution to explore the search space.

In [15]:
move_generators = [
    ChangeTimeSlot(),          # Move class to different time
    ChangeRoom(),               # Move class to different room
    SwapClasses(),              # Exchange time slots between two classes
    ChangeTimeSlotAndRoom(),    # Change both time and room
    FixRoomConflict(),          # Specialized: Fix room conflicts
    FixLecturerConflict(),      # Specialized: Fix lecturer conflicts
    FixRoomCapacity(),          # Specialized: Fix capacity issues
    FixMaxDailyPeriods(),       # Specialized: Fix daily period violations
    FixFridayPrayerConflict(),  # Specialized: Fix Friday prayer conflicts
    FixClassTypeTime(),         # Specialized: Fix class type time mismatches
    SwapFridayWithNonFriday(),  # Specialized: Swap Friday/non-Friday classes
]

print(f"\n🔀 Move Generators: {len(move_generators)}")
for gen in move_generators:
    print(f"   - {gen.name}")


🔀 Move Generators: 11
   - Change Time Slot
   - Change Room
   - Swap Classes
   - Change Time Slot and Room
   - Fix Room Conflict
   - Fix Lecturer Conflict
   - Fix Room Capacity
   - Fix Max Daily Periods
   - Fix Friday Prayer Conflict
   - Fix Class Type Time
   - Swap Friday With Non Friday


## 8. Configure Simulated Annealing

Set up the Simulated Annealing algorithm with all parameters.

In [40]:
config = SAConfig(
    initial_temperature=INITIAL_TEMPERATURE,
    min_temperature=MIN_TEMPERATURE,
    cooling_rate=COOLING_RATE,
    max_iterations=MAX_ITERATIONS,
    hard_constraint_weight=10000.0,
    tabu_search_enabled=TABU_SEARCH_ENABLED,
    tabu_tenure=TABU_TENURE,
    max_tabu_list_size=1000,
    reheating_threshold=REHEATING_THRESHOLD,
    reheating_factor=REHEATING_FACTOR,
    max_reheats=MAX_REHEATS,
    enable_intensification=ENABLE_INTENSIFICATION,
    intensification_iterations=INTENSIFICATION_ITERATIONS,
    max_intensification_attempts=MAX_INTENSIFICATION_ATTEMPTS,
    clone_state=lambda s: TimetableState(
        schedule=copy.deepcopy(s.schedule),
        available_time_slots=s.available_time_slots,
        rooms=s.rooms,
        lecturers=s.lecturers,
    ),
    logging=LoggingConfig(
        enabled=True,
        level="info",
        log_interval=1000,
        output="console",
    ),
)

print(f"\n⚙️  SA Configuration:")
print(f"   Temperature: {config.initial_temperature} → {config.min_temperature}")
print(f"   Cooling rate: {config.cooling_rate}")
print(f"   Max iterations: {config.max_iterations}")
print(f"   Tabu search: {config.tabu_search_enabled} (tenure={config.tabu_tenure})")
print(f"   Intensification: {config.enable_intensification}")


⚙️  SA Configuration:
   Temperature: 100000.0 → 1e-07
   Cooling rate: 0.9995
   Max iterations: 100000
   Tabu search: True (tenure=500)
   Intensification: True


## 9. Run Optimization

Execute the Simulated Annealing algorithm. This may take several minutes depending on your MAX_ITERATIONS setting.

**Note**: The optimization will log progress every 1000 iterations. Watch for:
- Phase 1: Eliminating hard constraint violations
- Phase 1.5: Intensification (if hard violations remain)
- Phase 2: Optimizing soft constraints
- Reheating events (when stuck in local optima)

In [17]:
print("\n" + "=" * 60)
print("🚀 Running Optimization...")
print("=" * 60)

sa = SimulatedAnnealing(state, all_constraints, move_generators, config)

start_time = time_python.time()
result = sa.solve()
elapsed_time = time_python.time() - start_time

print("\n" + "=" * 60)
print("✅ Optimization Complete!")
print("=" * 60)


🚀 Running Optimization...
[2026-01-11T02:28:43.670363] [INFO] Simulated Annealing initialized {'hard_constraints': 10, 'soft_constraints': 8, 'move_generators': 11, 'config': {'initial_temperature': 100000.0, 'min_temperature': 1e-07, 'cooling_rate': 0.9995, 'max_iterations': 100000}}
[2026-01-11T02:28:43.670492] [INFO] Starting optimization...
[2026-01-11T02:28:43.670511] [INFO] Phase 1: Eliminating hard constraint violations
[2026-01-11T02:28:43.694877] [INFO] Initial state {'fitness': '18795.81', 'hard_violations': 8}
[2026-01-11T02:29:42.272232] [INFO] [Phase 1] Iteration 1000: Temp = 66291.88, Hard violations = 5, Best = 5
[2026-01-11T02:31:22.828307] [INFO] [Phase 1] Iteration 3000: Temp = 29499.25, Hard violations = 4, Best = 4
[2026-01-11T02:32:14.280223] [INFO] [Phase 1] Iteration 4000: Temp = 19129.98, Hard violations = 4, Best = 4
[2026-01-11T02:33:05.055459] [INFO] [Phase 1] Iteration 5000: Temp = 12567.97, Hard violations = 2, Best = 2
[2026-01-11T02:33:32.890067] [INFO] 

## 10. Display Results

Analyze the optimization results including fitness, violations, and operator statistics.

In [18]:
print(f"\n📊 Results:")
print(f"   Final fitness: {result['fitness']:.4f}")
print(f"   Hard violations: {result['hard_violations']}")
print(f"   Soft violations: {result['soft_violations']}")
print(f"   Iterations: {result['iterations']}")
print(f"   Reheats: {result.get('reheats', 0)}")
print(f"   Execution time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

# Operator statistics
print(f"\n🔧 Operator Statistics:")
df_stats = pd.DataFrame(result['operator_stats']).T
df_stats.columns = ['Attempts', 'Improvements', 'Accepted', 'Success Rate']
df_stats['Success Rate'] = df_stats['Success Rate'].apply(lambda x: f"{x*100:.2f}%")
display(df_stats.sort_values('Attempts', ascending=False).style.hide(axis='index'))

# Constraint violations
print(f"\n📋 Constraint Violations:")
violations_summary = {}
for constraint in all_constraints:
    violations = constraint.get_violations(result['state'])
    if violations:
        violations_summary[constraint.name] = len(violations)

if violations_summary:
    df_violations = pd.DataFrame(list(violations_summary.items()),
                                  columns=['Constraint', 'Violations'])
    display(df_violations.sort_values('Violations', ascending=False).style.hide(axis='index'))
else:
    print("   No violations! Perfect schedule! 🎉")


📊 Results:
   Final fitness: 61.5674
   Hard violations: 0
   Soft violations: 129
   Iterations: 81348
   Reheats: 3
   Execution time: 4369.65 seconds (72.83 minutes)

🔧 Operator Statistics:


Attempts,Improvements,Accepted,Success Rate
18140.000000,1450.000000,8148.000000,7.99%
15717.000000,127.000000,4494.000000,0.81%
15486.000000,1012.000000,8080.000000,6.53%
11152.000000,6.000000,9912.000000,0.05%
1043.000000,57.000000,64.000000,5.47%
671.000000,8.000000,283.000000,1.19%
117.000000,11.000000,110.000000,9.40%
2.000000,2.000000,2.000000,100.00%
1.000000,1.000000,1.000000,100.00%
0.000000,0.000000,0.000000,0.00%



📋 Constraint Violations:


Constraint,Violations
Research Day,51
Preferred Time,22
Evening Class Priority,17
Compactness,16
Prayer Time Overlap,15
Preferred Room,6
Overflow Penalty,2


## 11. Save Results to JSON

Export the final schedule to a JSON file for further analysis or integration.

In [21]:
final_state = result['state']

# Create flattened version for display and nested version for export
schedule_result_flat = []
schedule_result_nested = []

for entry in final_state.schedule:
    end_time, prayer_time_added = calculate_end_time(
        entry.time_slot.start_time, entry.sks, entry.time_slot.day
    )
    is_overflow = not entry.needs_lab and 'lab' in entry.room.lower()

    # Flattened version for display
    schedule_result_flat.append({
        "classId": entry.class_id,
        "className": entry.class_name,
        "class": entry.kelas,
        "prodi": entry.prodi,
        "lecturers": entry.lecturers,
        "room": entry.room,
        "day": entry.time_slot.day,
        "period": entry.time_slot.period,
        "startTime": entry.time_slot.start_time,
        "endTime": entry.time_slot.end_time,
        "sks": entry.sks,
        "needsLab": entry.needs_lab,
        "participants": entry.participants,
        "classType": entry.class_type,
        "prayerTimeAdded": prayer_time_added,
        "isOverflowToLab": is_overflow,
    })
    
    # Nested version for JSON export (matches TypeScript format)
    schedule_result_nested.append({
        "classId": entry.class_id,
        "className": entry.class_name,
        "class": entry.kelas,
        "prodi": entry.prodi,
        "lecturers": entry.lecturers,
        "room": entry.room,
        "timeSlot": {
            "period": entry.time_slot.period,
            "day": entry.time_slot.day,
            "startTime": entry.time_slot.start_time,
            "endTime": entry.time_slot.end_time,
        },
        "sks": entry.sks,
        "needsLab": entry.needs_lab,
        "participants": entry.participants,
        "classType": entry.class_type,
        "prayerTimeAdded": prayer_time_added,
        "isOverflowToLab": is_overflow,
    })

final_result = {
    "fitness": result['fitness'],
    "hardViolations": result['hard_violations'],
    "softViolations": result['soft_violations'],
    "iterations": result['iterations'],
    "schedule": schedule_result_nested,
}

OUTPUT_PATH = "/home/emmanuelabayor/projects/timetable-sa/python/timetable-result.json"
with open(OUTPUT_PATH, 'w') as f:
    json.dump(final_result, f, indent=2, ensure_ascii=False)

print(f"\n💾 Results saved to: {OUTPUT_PATH}")
print(f"   Schedule entries: {len(schedule_result_nested)}")


💾 Results saved to: /home/emmanuelabayor/projects/timetable-sa/python/timetable-result.json
   Schedule entries: 356


## 12. Sample Schedule Display

Display the generated schedule in a readable format.

In [30]:
# Create DataFrame from schedule (using flattened version from previous cell)
df_schedule = pd.DataFrame(schedule_result_flat)

# Display first 10 entries
print(f"\n📅 Sample Schedule (first 10 entries):")
display_cols = ['classId', 'className', 'class', 'day', 'startTime', 'endTime', 'room', 'lecturers']
df_display = df_schedule[display_cols].head(10)
display(df_display.style.hide(axis='index'))

# Display schedule grouped by day
print(f"\n📅 Schedule by Day:")
for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]:
    day_schedule = df_schedule[df_schedule['day'] == day].sort_values('startTime')
    if len(day_schedule) > 0:
        print(f"\n### {day} ({len(day_schedule)} classes)")
        display_cols = ['startTime', 'endTime', 'classId', 'className', 'room']
        display(day_schedule[display_cols].head(10).style.hide(axis='index'))


📅 Sample Schedule (first 10 entries):


classId,className,class,day,startTime,endTime,room,lecturers
VD13IE53,User Interface & User Experience,DKV-7A,Tuesday,07:30,10:00,G5-Lab1,['MNR']
EM13LA02,Aljabar Linear,MR-3-SIG,Thursday,07:30,09:10,G2-R5,['AAV']
LE13DA13,ANALITIKA DATA,TL-5B,Wednesday,08:20,10:50,G2-R7,['PMN']
CE11SB03,Pilihan Keahlian 2 Material Canggih dan Teknik Karakterisasi,TK - 7A,Tuesday,12:30,15:00,G2-R3,['UAG']
LE13DA13,ANALITIKA DATA,TL-5B,Thursday,10:00,12:30,B3-R1,['PMN']
VD13EX03,Exhibition,DKV-7A,Thursday,12:30,15:00,CM-204,['DAP']
CE13OL11,Praktikum Kimia Organik,TK - 3A,Friday,08:20,09:10,B3-R2,['SII']
EM13TM03,Teori dan Metodologi Perancangan,"MR-5A, MR-5S",Friday,13:20,16:20,CM-207,['NWF']
MG13MP13,Manajamen Pemasaran,MG-3B,Wednesday,12:30,15:00,CM-203,['ANW']
DT13MD13,Matematika Diskret,IF-1A,Tuesday,12:30,15:00,B3-R1,['NGT']



📅 Schedule by Day:

### Monday (73 classes)


startTime,endTime,classId,className,room
07:30,09:10,SE13IC02,Islamic Corporate Governance & CSR,G2-R7
07:30,09:10,AB13A402,"Lingkungan, Sosial, dan Tata Kelola",CM-101
07:30,10:00,MG13SI03,Sistem Informasi Manajemen,G4-R3
07:30,09:10,MG13SB22,Seminar Bisnis,CM-103
07:30,10:00,SE13CB53,Perilaku Konsumen,G2-R5
07:30,10:00,MG13PM03,Pengantar Manajemen,CM-LabVirtual
07:30,10:00,VD13KM53,Komik,CM-208
07:30,10:00,DT13ST13,Statistika Dasar,G2-R4
07:30,10:00,IF13PP13,Proyek Pengembangan Perangkat Lunak,CM-Lab3
07:30,10:00,IS13SD53,Sistem Dinamik,CM-206



### Tuesday (75 classes)


...

startTime,endTime,classId,className,room
07:30,09:10,EM13LA02,Aljabar Linear,G2-R5
07:30,10:00,MM23PM03,Project Management,G2-R2
07:30,10:00,AB13B413,Manajemen Risiko,CM-LabVirtual
07:30,10:00,MG13MS23,Manajemen Strategi,CM-201
07:30,09:10,GS13IL02,Bahasa Indonesia,G4-R3
07:30,10:00,VD13IE53,User Interface & User Experience,G5-Lab2
07:30,10:00,IS13MP13,Metodologi Penelitian & Penulisan Ilmiah,G2-R7
07:30,09:10,LE13TD02,MENGGAMBAR TEKNIK,CM-Lab3
07:30,10:00,MG13PB03,Pengantar Bisnis dan Korporasi,CM-204
07:30,10:00,IF13AM13,Pemrograman Aplikasi Mobile,G5-Lab1



### Friday (47 classes)


startTime,endTime,classId,className,room
07:30,10:00,ET13CA03,Kalkulus I,G4-R2
07:30,10:00,AB13A803,Manajemen Keuangan,G2-R3
07:30,10:50,IF13P114,Pemrograman 1,G5-LabAudioVisual
07:30,09:10,GS13RG02,Agama Islam Agama Katolik Agama Protestan Agama Hindu Agama Budha Agama Kong Hu Cu,B3-R3
07:30,10:00,VD13DK03,Desain Kemasan,CM-202
07:30,08:20,CE13OL11,Praktikum Kimia Organik,G5-Lab1
07:30,09:10,CE11EE02,Ekonomi Teknik,G2-R5
07:30,10:00,MG13ME13,"Manajemen Pertemuan, Eksposisi, Even, dan Konvensi",CM-103
07:30,10:00,EM13MR13,Riset Pasar dan Pemasaran,B2-R1
07:30,09:10,IS13JK02,Jaringan Komputer,CM-Lab3



### Saturday (5 classes)


startTime,endTime,classId,className,room
09:10,11:40,MM23SE03,Sustainability and Enviromental Management,G2-R2
10:00,12:30,MM23SM03,Strategic Marketing Management,B3-R1
13:20,16:20,MM23SE03,Sustainability and Enviromental Management,B3-R2
13:20,16:20,MM23CF03,Corporate Finance,B3-R1
18:00,21:00,MM23AB03,Agile and Disruptive Business Strategy,B3-R2


## 13. Visualization by Class (Kelas)

This section displays the complete schedule for each class (Kelas). Each class gets its own "card" showing:

- **Class name**: The class identifier (e.g., IF-1, IF-2, IF-3)
- **Program (Prodi)**: The academic program
- **Total classes**: Number of scheduled courses
- **Schedule table**: Complete weekly schedule with day, time, course, room, and lecturer

**Note**: Combined classes (e.g., "IF-1A, IF-3B") are automatically split into separate cards for each class.

In [31]:
# Import HTML display for card visualization
from IPython.display import HTML, display

# Function to extract individual classes from combined class strings
def expand_classes(class_str):
    """Expand combined class names like 'IF-1A, IF-3B' into individual classes."""
    classes = [c.strip() for c in str(class_str).split(',')]
    return classes

# Create expanded DataFrame with one row per class
expanded_rows = []
for _, row in df_schedule.iterrows():
    classes = expand_classes(row['class'])
    for cls in classes:
        expanded_rows.append({
            'class': cls,
            'classId': row['classId'],
            'className': row['className'],
            'prodi': row['prodi'],
            'day': row['day'],
            'startTime': row['startTime'],
            'endTime': row['endTime'],
            'room': row['room'],
            'sks': row['sks'],
            'lecturers': row['lecturers']
        })

df_by_class = pd.DataFrame(expanded_rows)

# Get unique classes sorted
unique_classes = sorted(df_by_class['class'].unique())

print(f"\n📚 Schedule by Class ({len(unique_classes)} classes):")
print(f"   Showing first 10 classes (scroll to see more)")

# Display function for class schedule card
def display_class_schedule(class_name):
    """Display a formatted schedule card for a specific class."""
    class_data = df_by_class[df_by_class['class'] == class_name].sort_values(['day', 'startTime'])

    if len(class_data) == 0:
        return f"<div style='border:1px solid #ddd; padding:15px; margin:10px 0; border-radius:8px;'><h3>📚 {class_name}</h3><p>No classes scheduled.</p></div>"

    # Get class info from first row
    first_row = class_data.iloc[0]
    prodi = first_row['prodi']

    html = f"""
    <div style='border:1px solid #4CAF50; padding:20px; margin:15px 0; border-radius:10px; background:linear-gradient(135deg, #f8fff8 0%, #e8f5e9 100%); box-shadow:0 4px 6px rgba(0,0,0,0.1);'>
        <h2 style='color:#2E7D32; margin-top:0;'>📚 {class_name}</h2>
        <p style='color:#666; font-style:italic;'>{prodi}</p>
        <p><strong>Total Classes:</strong> {len(class_data)}</p>
        <table style='width:100%; border-collapse:collapse; margin-top:15px;'>
            <thead>
                <tr style='background:#4CAF50; color:white;'>
                    <th style='padding:10px; text-align:left; border:1px solid #45a049;'>Day</th>
                    <th style='padding:10px; text-align:left; border:1px solid #45a049;'>Time</th>
                    <th style='padding:10px; text-align:left; border:1px solid #45a049;'>Course</th>
                    <th style='padding:10px; text-align:left; border:1px solid #45a049;'>Room</th>
                    <th style='padding:10px; text-align:left; border:1px solid #45a049;'>Lecturer</th>
                </tr>
            </thead>
            <tbody>
    """

    for _, row in class_data.iterrows():
        html += f"""
                <tr style='border-bottom:1px solid #ddd;'>
                    <td style='padding:10px; border:1px solid #ddd;'>{row['day']}</td>
                    <td style='padding:10px; border:1px solid #ddd;'>{row['startTime']} - {row['endTime']}</td>
                    <td style='padding:10px; border:1px solid #ddd;'>
                        <strong>{row['classId']}</strong><br/>
                        <small style='color:#666;'>{row['className']}</small>
                    </td>
                    <td style='padding:10px; border:1px solid #ddd;'>{row['room']}</td>
                    <td style='padding:10px; border:1px solid #ddd;'>{', '.join(row['lecturers'])}</td>
                </tr>
        """

    html += """
            </tbody>
        </table>
    </div>
    """
    return html

# Display first 10 classes
for cls in unique_classes[:10]:
    display(HTML(display_class_schedule(cls)))

if len(unique_classes) > 10:
    print(f"\n... and {len(unique_classes) - 10} more classes")


📚 Schedule by Class (65 classes):
   Showing first 10 classes (scroll to see more)


Day,Time,Course,Room,Lecturer
Monday,07:30 - 09:10,ET13PT02 Teori Probabilitas,G2-R3,CWR
Monday,10:50 - 12:30,ET13PT02 Teori Probabilitas,G3-R1,CWR
Tuesday,10:00 - 12:30,ET13SC13 Manajemen Rantai Pasok,G5-Lab1,AAV
Wednesday,07:30 - 09:10,EM13LE02 Legal dan Etika Profesi,B3-R1,NWF


Day,Time,Course,Room,Lecturer
Monday,08:20 - 10:50,GS13EL03 Bahasa Inggris,G4-R4,IKN
Monday,13:20 - 16:20,AB13A503 Akuntansi Dasar,G4-R2,FRD
Thursday,10:00 - 12:30,AB13A303 Statistik Bisnis,CM-205,LRS
Tuesday,10:00 - 12:30,AB13A103 Ekonomi Bisnis,CM-102,AWD
Tuesday,13:20 - 16:20,AC130703 Pengantar Bisnis,CM-204,AWD


Day,Time,Course,Room,Lecturer
Friday,08:20 - 10:50,AC131603 Tata Kelola dan Etika Profesi,CM-205,ELD
Monday,10:00 - 12:30,AB13A803 Manajemen Keuangan,CM-208,FRD
Monday,13:20 - 16:20,AB13B413 Manajemen Risiko,G4-R3,AWD
Thursday,08:20 - 10:50,AB13A713 Manajemen Operasional,CM-103,MAK
Tuesday,10:00 - 12:30,AB13A803 Manajemen Keuangan,CM-201,FRD
Tuesday,12:30 - 15:00,AC131313 Akuntansi Keuangan Menengah 1,CM-101,ADL
Wednesday,07:30 - 10:00,AB13A803 Manajemen Keuangan,CM-102,FRD
Wednesday,10:00 - 12:30,AC131413 Akuntansi Manajemen,CM-202,ADL
Wednesday,12:30 - 15:00,AB13A803 Manajemen Keuangan,G4-R4,FRD


Day,Time,Course,Room,Lecturer
Friday,08:20 - 10:50,AC133533 Akuntansi Keuangan Lanjutan 2,CM-201,ADL
Monday,10:00 - 12:30,AC133013 Teori Akuntansi,CM-103,AFT
Thursday,07:30 - 10:00,AB13B413 Manajemen Risiko,CM-LabVirtual,FRD
Thursday,10:00 - 12:30,AB13B413 Manajemen Risiko,CM-201,FRD
Tuesday,07:30 - 10:00,AC132833 Akuntansi Keuangan Lanjutan 1,G4-R3,AFT
Tuesday,10:00 - 12:30,AC133113 Analitika Data,CM-206,MAK
Tuesday,12:30 - 15:00,AC133223 Simulasi Kelayakan Bisnis,CM-LabVirtual,MAK
Wednesday,07:30 - 10:00,AC133633 Metodologi Penelitian,CM-101,ELD
Wednesday,13:20 - 16:20,AC133633 Metodologi Penelitian,CM-205,ELD


...

Day,Time,Course,Room,Lecturer
Friday,08:20 - 10:50,VD13MP03 Metode Reprografika,CM-102,DAP
Monday,09:10 - 11:40,VD13RI03 Budaya Rupa Indonesia,G2-R6,RNA
Monday,12:30 - 16:20,VD13DI04 DKV I Desain Informasi,G2-R6,RNA
Thursday,07:30 - 10:00,VD13SK03 Sketsa,B3-R3,DAP
Thursday,10:00 - 12:30,VD13FD03 Fotografi Dasar,G5-Lab1,MNR
Thursday,12:30 - 15:00,VD13FD03 Fotografi Dasar,G5-Lab2,MNR
Tuesday,10:00 - 12:30,VD13TG03 Tipografi,G3-R2,TAN
Tuesday,12:30 - 14:10,GS13IL02 Bahasa Indonesia,G4-R3,ARI
Wednesday,09:10 - 11:40,VD13TG03 Tipografi,G2-R6,TAN


Day,Time,Course,Room,Lecturer
Friday,07:30 - 10:00,VD13DK03 Desain Kemasan,CM-202,RNA
Monday,09:10 - 11:40,VD13CW03 Copywriting,G4-R1,RNU
Thursday,09:10 - 12:30,VD13JN14 DKV III Jenama,CM-208,TAN
Thursday,14:10 - 17:10,VD13DK03 Desain Kemasan,CM-208,RNA


Day,Time,Course,Room,Lecturer
Friday,08:20 - 10:50,VD13SM03 Seminar dan Metodologi Penelitian,CM-LabVirtual,RNU
Monday,07:30 - 10:00,VD13KM53 Komik,CM-208,DAP
Monday,13:20 - 16:20,VD13IE53 User Interface & User Experience,G5-Lab1,MNR
Thursday,07:30 - 10:00,VD13IE53 User Interface & User Experience,G5-Lab2,MNR
Thursday,12:30 - 15:00,VD13EX03 Exhibition,CM-204,DAP
Tuesday,07:30 - 10:00,VD13IE53 User Interface & User Experience,G5-Lab1,MNR
Tuesday,10:00 - 12:30,VD13EX03 Exhibition,CM-203,DAP
Tuesday,12:30 - 15:00,VD13BS53 Perencanaan Bisnis,G4-R2,TAN
Wednesday,07:30 - 10:00,VD13SA53 Street Art,CM-207,DAP
Wednesday,10:00 - 12:30,VD13HC03 Digital Heritage Conservation,G4-R4,DAP


Day,Time,Course,Room,Lecturer
Friday,13:20 - 15:00,SE13ME02 Sejarah Pemikiran Ekonomi Islam,G5-Lab2,MHA
Monday,08:20 - 10:50,AB13A503 Akuntansi Dasar,G3-R2,ADM
Thursday,10:00 - 12:30,AB13A503 Akuntansi Dasar,G2-R4,ADM
Thursday,13:20 - 16:20,SE13MI03 Pengantar Manajemen dan Bisnis Islam,B3-R2,MAL
Thursday,18:00 - 21:00,GS13EL03 Bahasa Inggris,B3-R2,MAF
Tuesday,10:00 - 12:30,AB13A103 Ekonomi Bisnis,B2-R1,EMH
Tuesday,12:30 - 14:10,GS13IL02 Bahasa Indonesia,G4-R3,ARI



... and 55 more classes


In [ ]:
# Interactive class selector using ipywidgets
from ipywidgets import interact, Dropdown

def show_class_schedule(class_name):
    """Display schedule card for selected class."""
    display(HTML(display_class_schedule(class_name)))

print("\n🔍 Select a class to view its schedule:")
interact(show_class_schedule, 
         class_name=Dropdown(
             options=sorted(unique_classes), 
             value=sorted(unique_classes)[0],
             description='Class:'
         ))


🔍 Select a class to view its schedule:


interactive(children=(Dropdown(description='Class:', options=('7-SIG', 'AK-1A', 'AK-3A', 'AK-5A', 'AK-7A', 'DK…

<function __main__.show_class_schedule(class_name)>

## 14. Visualization by Lecturer (Dosen)

This section displays the complete teaching schedule for each lecturer. Each lecturer gets their own "card" showing:

- **Lecturer name**: The full name of the lecturer
- **Lecturer code**: The lecturer's identifier code
- **Program (Prodi)**: The academic program they belong to
- **Total Classes**: Number of courses they teach
- **Total SKS**: Sum of credit hours (Satuan Kredit Semester)
- **Schedule table**: Complete weekly schedule with day, time, course, class, room, and participants

In [33]:
# Expand by lecturer - create one row per lecturer per course
expanded_rows_lecturer = []
for _, row in df_schedule.iterrows():
    for lecturer in row['lecturers']:
        expanded_rows_lecturer.append({
            'lecturer': lecturer,
            'classId': row['classId'],
            'className': row['className'],
            'class': row['class'],
            'prodi': row['prodi'],
            'day': row['day'],
            'startTime': row['startTime'],
            'endTime': row['endTime'],
            'room': row['room'],
            'sks': row['sks'],
            'participants': row['participants']
        })

df_by_lecturer = pd.DataFrame(expanded_rows_lecturer)

# Get unique lecturers sorted
unique_lecturers = sorted(df_by_lecturer['lecturer'].unique())

# Get lecturer details from original data
lecturer_details = {l.code: l for l in lecturers}

print(f"\n👨‍🏫 Schedule by Lecturer ({len(unique_lecturers)} lecturers):")
print(f"   Showing first 10 lecturers (scroll to see more)")

# Display function for lecturer schedule card
def display_lecturer_schedule(lecturer_code):
    """Display a formatted schedule card for a specific lecturer."""
    lecturer_data = df_by_lecturer[df_by_lecturer['lecturer'] == lecturer_code].sort_values(['day', 'startTime'])

    # Get lecturer details
    details = lecturer_details.get(lecturer_code)
    if details:
        lecturer_name = details.name
        lecturer_prodi = details.prodi
    else:
        lecturer_name = lecturer_code
        lecturer_prodi = "N/A"

    if len(lecturer_data) == 0:
        return f"<div style='border:1px solid #ddd; padding:15px; margin:10px 0; border-radius:8px;'><h3>👨‍🏫 {lecturer_name}</h3><p>No classes assigned.</p></div>"

    # Calculate total SKS
    total_sks = lecturer_data['sks'].sum()

    html = f"""
    <div style='border:1px solid #2196F3; padding:20px; margin:15px 0; border-radius:10px; background:linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%); box-shadow:0 4px 6px rgba(0,0,0,0.1);'>
        <h2 style='color:#1565C0; margin-top:0;'>👨‍🏫 {lecturer_name}</h2>
        <p style='color:#666;'><strong>Code:</strong> {lecturer_code} | <strong>Program:</strong> {lecturer_prodi}</p>
        <p><strong>Total Classes:</strong> {len(lecturer_data)} | <strong>Total SKS:</strong> {total_sks}</p>
        <table style='width:100%; border-collapse:collapse; margin-top:15px;'>
            <thead>
                <tr style='background:#2196F3; color:white;'>
                    <th style='padding:10px; text-align:left; border:1px solid #1976D2;'>Day</th>
                    <th style='padding:10px; text-align:left; border:1px solid #1976D2;'>Time</th>
                    <th style='padding:10px; text-align:left; border:1px solid #1976D2;'>Course</th>
                    <th style='padding:10px; text-align:left; border:1px solid #1976D2;'>Class</th>
                    <th style='padding:10px; text-align:left; border:1px solid #1976D2;'>Room</th>
                    <th style='padding:10px; text-align:left; border:1px solid #1976D2;'>Participants</th>
                </tr>
            </thead>
            <tbody>
    """

    for _, row in lecturer_data.iterrows():
        html += f"""
                <tr style='border-bottom:1px solid #ddd;'>
                    <td style='padding:10px; border:1px solid #ddd;'>{row['day']}</td>
                    <td style='padding:10px; border:1px solid #ddd;'>{row['startTime']} - {row['endTime']}</td>
                    <td style='padding:10px; border:1px solid #ddd;'>
                        <strong>{row['classId']}</strong><br/>
                        <small style='color:#666;'>{row['className']}</small>
                    </td>
                    <td style='padding:10px; border:1px solid #ddd;'>{row['class']}</td>
                    <td style='padding:10px; border:1px solid #ddd;'>{row['room']}</td>
                    <td style='padding:10px; border:1px solid #ddd;'>{row['participants']}</td>
                </tr>
        """

    html += """
            </tbody>
        </table>
    </div>
    """
    return html

# Display first 10 lecturers
for lec in unique_lecturers[:10]:
    display(HTML(display_lecturer_schedule(lec)))

if len(unique_lecturers) > 10:
    print(f"\n... and {len(unique_lecturers) - 10} more lecturers")


👨‍🏫 Schedule by Lecturer (67 lecturers):
   Showing first 10 lecturers (scroll to see more)


Day,Time,Course,Class,Room,Participants
Friday,07:30 - 10:00,ET13CA03 Kalkulus I,MR-1A,G4-R2,40
Monday,10:00 - 11:40,EM13LA02 Aljabar Linear,MR-3A,B3-R2,30
Thursday,07:30 - 09:10,EM13LA02 Aljabar Linear,MR-3-SIG,G2-R5,15
Thursday,10:00 - 11:40,EM13LA02 Aljabar Linear,MR-3-SIG,B2-R1,15
Thursday,13:20 - 15:00,EM13LA02 Aljabar Linear,MR-3-SIG,G2-R2,15
Tuesday,10:00 - 12:30,ET13SC13 Manajemen Rantai Pasok,"MR-3-SIG,MR-5S,7-SIG",G5-Lab1,20
Tuesday,15:30 - 18:00,ET13CA03 Kalkulus I,MR-1S,B2-R1,10


Day,Time,Course,Class,Room,Participants
Friday,07:30 - 09:10,IS13JK02 Jaringan Komputer,SI-1A,CM-Lab3,45
Friday,13:20 - 15:00,IS13JK02 Jaringan Komputer,SI-1A,CM-Lab3,45
Monday,09:10 - 11:40,IS13VA33 Visualisasi dan Analisis Data,SI-3A,G5-Lab1,30
Monday,14:10 - 17:10,IS13VA33 Visualisasi dan Analisis Data,SI-5A,CM-LabVirtual,44
Thursday,10:00 - 12:30,IS13VA33 Visualisasi dan Analisis Data,SI-3B,G5-Lab2,30
Thursday,13:20 - 16:20,IS13PL33 Pengembangan Perangkat Lunak,"SI-3A,SI-3B",G4-R4,56
Tuesday,07:30 - 10:00,IS13VA33 Visualisasi dan Analisis Data,SI-3A,CM-206,30
Tuesday,12:30 - 15:00,IF13UP13 Pengujian Perangkat Lunak,"IF-7A,IF-7B",G4-R4,60
Wednesday,10:00 - 11:40,IS13JK02 Jaringan Komputer,SI-1B,CM-LabVirtual,45
Wednesday,12:30 - 14:10,IS13JK02 Jaringan Komputer,SI-1A,CM-Lab3,45


Day,Time,Course,Class,Room,Participants
Friday,08:20 - 10:50,AC133533 Akuntansi Keuangan Lanjutan 2,AK-5A,CM-201,42
Thursday,13:20 - 16:20,AB13A503 Akuntansi Dasar,MG-1B,CM-103,45
Tuesday,12:30 - 15:00,AC131313 Akuntansi Keuangan Menengah 1,AK-3A,CM-101,40
Wednesday,10:00 - 12:30,AC131413 Akuntansi Manajemen,AK-3A,CM-202,40


Day,Time,Course,Class,Room,Participants
Friday,13:20 - 16:20,SE13IA33 Akuntansi Syariah,ES-3A,B3-R2,30
Monday,08:20 - 10:50,AB13A503 Akuntansi Dasar,ES-1A,G3-R2,25
Monday,12:30 - 15:00,SE13BF43 Studi Kelayanan Bisnis Islam,ES-5A,B2-R1,25
Thursday,10:00 - 12:30,AB13A503 Akuntansi Dasar,ES-1A,G2-R4,25
Tuesday,10:00 - 12:30,SE13IB03 Pengantar Bank dan Industri Keuangan Non-Bank Syariah,ES-3A,G4-R4,30
Wednesday,07:30 - 10:00,SE13BF43 Studi Kelayanan Bisnis Islam,ES-5A,G5-Lab1,25
Wednesday,13:20 - 16:20,SE13BF43 Studi Kelayanan Bisnis Islam,ES-5A,G3-R2,25


...

Day,Time,Course,Class,Room,Participants
Thursday,07:30 - 09:10,ET13PT02 Teori Probabilitas,"MR-3A,MR-5A,MR-7A",G4-R4,42
Thursday,12:30 - 14:10,ET13PT02 Teori Probabilitas,"MR-3A,MR-5A,MR-7A",CM-207,42
Tuesday,10:00 - 11:40,ET13PT02 TEORI PROBABILITAS,TL-1A,CM-204,35
Tuesday,12:30 - 14:10,ET13PT02 TEORI PROBABILITAS,TL-1A,CM-208,35


Day,Time,Course,Class,Room,Participants
Monday,10:00 - 12:30,EM13CP03 Pemrograman Komputer,MR-1A,CM-Lab3,40
Monday,18:00 - 21:00,EM13CP03 Pemrograman Komputer,MR-1S,B2-R1,10


Day,Time,Course,Class,Room,Participants
Friday,08:20 - 10:50,MG13PD13 Pemasaran Digital,MG-5B,CM-206,45
Saturday,10:00 - 12:30,MM23SM03 Strategic Marketing Management,MM-1A,B3-R1,15
Thursday,07:30 - 10:00,MG13PB03 Pengantar Bisnis dan Korporasi,MG-1A,CM-204,45
Thursday,10:00 - 12:30,MG12FB53 Manajemen Bisnis Keluarga (Family Business),MG-7A,CM-LabVirtual,34
Tuesday,10:00 - 12:30,MG13PB03 Pengantar Bisnis dan Korporasi,MG-1B,CM-205,45
Wednesday,10:00 - 12:30,MG13PD13 Pemasaran Digital,MG-5D,CM-Lab3,40


Day,Time,Course,Class,Room,Participants
Monday,07:30 - 09:10,MG13SB22 Seminar Bisnis,MG-7C,CM-103,34
Monday,13:20 - 16:20,MG12MP53 Manajemen Penjualan,MG-7A,CM-103,34
Thursday,12:30 - 14:10,MG13SB22 Seminar Bisnis,MG-7B,CM-101,34
Tuesday,08:20 - 10:00,MG13SB22 Seminar Bisnis,MG-7A,CM-Lab3,34
Tuesday,10:00 - 12:30,MG13MP13 Manajamen Pemasaran,MG-3B,CM-101,45
Tuesday,12:30 - 15:00,MG13MO03 Manajemen Operasi,MG-3A,CM-205,45
Wednesday,10:00 - 12:30,MG13PD13 Pemasaran Digital,MG-5C,CM-102,40
Wednesday,12:30 - 15:00,MG13MP13 Manajamen Pemasaran,MG-3B,CM-203,45



... and 57 more lecturers


In [ ]:
# Interactive lecturer selector using ipywidgets
def show_lecturer_schedule(lecturer_code):
    """Display schedule card for selected lecturer."""
    display(HTML(display_lecturer_schedule(lecturer_code)))

print("\n🔍 Select a lecturer to view their schedule:")
interact(show_lecturer_schedule, 
         lecturer_code=Dropdown(
             options=sorted(unique_lecturers), 
             value=sorted(unique_lecturers)[0],
             description='Lecturer:'
         ))

## 15. Summary Statistics

This section provides summary statistics about class and lecturer workloads, helping to analyze the distribution of teaching responsibilities across the institution.

In [34]:
print("\n📊 Summary Statistics:")
print("=" * 60)

# Class workload statistics
print(f"\n📚 Class Workload:")
class_workload = df_by_class.groupby('class').size().sort_values(ascending=False)
print(f"   Total unique classes: {len(class_workload)}")
print(f"   Busiest Class: {class_workload.index[0]} ({class_workload.iloc[0]} classes)")
print(f"   Least Busy Class: {class_workload.index[-1]} ({class_workload.iloc[-1]} classes)")
print(f"   Average Classes per Group: {class_workload.mean():.1f}")
print(f"   Median: {class_workload.median():.1f}")
print(f"   Std Dev: {class_workload.std():.1f}")

# Lecturer workload statistics
print(f"\n👨‍🏫 Lecturer Workload:")
lecturer_workload = df_by_lecturer.groupby('lecturer').agg({
    'classId': 'count',
    'sks': 'sum',
    'participants': 'sum'
}).rename(columns={'classId': 'num_classes', 'sks': 'total_sks', 'participants': 'total_students'})
lecturer_workload = lecturer_workload.sort_values('num_classes', ascending=False)

print(f"   Total unique lecturers: {len(lecturer_workload)}")
print(f"   Busiest Lecturer: {lecturer_workload.index[0]} ({lecturer_workload.iloc[0]['num_classes']} classes, {lecturer_workload.iloc[0]['total_sks']} SKS)")
print(f"   Average Classes per Lecturer: {lecturer_workload['num_classes'].mean():.1f}")
print(f"   Average SKS per Lecturer: {lecturer_workload['total_sks'].mean():.1f}")
print(f"   Median SKS per Lecturer: {lecturer_workload['total_sks'].median():.1f}")

# Display top 5 lecturers by workload
print(f"\n👨‍🏫 Top 5 Lecturers by Workload:")
top_lecturers = lecturer_workload.head(5)
for idx, row in top_lecturers.iterrows():
    details = lecturer_details.get(idx)
    name = details.name if details else idx
    print(f"   {idx} ({name}): {int(row['num_classes'])} classes, {int(row['total_sks'])} SKS, {int(row['total_students'])} total students")

# Day distribution
print(f"\n📅 Class Distribution by Day:")
day_dist = df_schedule['day'].value_counts()
for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]:
    count = day_dist.get(day, 0)
    print(f"   {day}: {count} classes")

# Room utilization
print(f"\n🏠 Room Utilization:")
room_util = df_schedule['room'].value_counts().head(10)
print(f"   Top 10 Most Used Rooms:")
for room, count in room_util.items():
    print(f"      {room}: {count} classes")

print("=" * 60)


📊 Summary Statistics:

📚 Class Workload:
   Total unique classes: 65
   Busiest Class: DKV-7A (11 classes)
   Least Busy Class: SI-5B (1 classes)
   Average Classes per Group: 6.1
   Median: 6.0
   Std Dev: 2.6

👨‍🏫 Lecturer Workload:
   Total unique lecturers: 67
   Busiest Lecturer: FKH (12 classes, 32 SKS)
   Average Classes per Lecturer: 5.9
   Average SKS per Lecturer: 16.1
   Median SKS per Lecturer: 16.0

👨‍🏫 Top 5 Lecturers by Workload:
   FKH (Fara Kamila Hudy, S.T., M.T.): 12 classes, 32 SKS, 345 total students
   CWR (Citra Wahyu Rizkita, S.T., M.T.): 11 classes, 28 SKS, 382 total students
   ADF (Ardhi Dwi Firmansyah, S.Kom., M.Kom.): 10 classes, 26 SKS, 430 total students
   SII (Surya Iryana Ihsanpuro, S.T., M.T.): 10 classes, 19 SKS, 388 total students
   UAG (Dr. Eng. Ufafa Anggarini, S.Si., M.Si.): 10 classes, 24 SKS, 364 total students

📅 Class Distribution by Day:
   Monday: 73 classes
   Tuesday: 75 classes
   Wednesday: 74 classes
   Thursday: 82 classes
   Friday